In [5]:
#####設定讀取資料的年份與時段#####

year = 104    #110年或104年
rush_hour = '昏峰'    #'晨峰'或'昏峰'

inputLocation = f"{year}_現況_{rush_hour}/"

################################

import pandas as pd

df= pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]
if rush_hour == '晨峰':
    real_flow = df["晨峰實際流量"]
elif rush_hour == '昏峰':
    real_flow = df["昏峰實際流量"]
print(f'——————————————————————校估{year}年{rush_hour}指派流量——————————————————————')

real_flow = real_flow.set_index("group")


ue_results = pd.read_csv(inputLocation + f"{year}_現況_{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)


#look up UE_flow in ue_results to add a new column to sheet1 and sheet2
#for sheet2, we need to find the corresponding link name for A1, B1 and A2, B2
#that is, there are two link to be found for each row in sheet2
#then we need to add up the two link's UE_flow to get the total flow for this row
#so that we can compare with AM flow
sheet1["link_name1"] = sheet1.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
#calculate the total flow and capacity for this row

#reserve the link,AM,UE_flow
sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

group_names = sheet1['group'].unique()
groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in group_names}
for index, row in sheet1.iterrows():
    group = row['group']
    groups[group]['UE_flow_1'] += row['UE_flow_1']
    groups[group]['UE_flow_2'] += row['UE_flow_2']

# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
# {real_flow.loc[group, 'name']}
for group, flows in groups.items():
    print(f"""【{group}】 {real_flow.loc[group, 'name']}
    UE flow 往南/東 = {round(flows['UE_flow_1'])}， model diff = {(flows['UE_flow_1'] - real_flow.loc[group, '南/東']) / real_flow.loc[group, '南/東']* 100:.2f}%， CUBE diff = {(real_flow.loc[group, 'CUBE南/東'] - real_flow.loc[group, '南/東']) / real_flow.loc[group, '南/東']* 100:.2f}%
    UE flow 往北/西 = {round(flows['UE_flow_2'])}， model diff = {(flows['UE_flow_2'] - real_flow.loc[group, '北/西']) / real_flow.loc[group, '北/西']* 100:.2f}%， CUBE diff = {(real_flow.loc[group, 'CUBE北/西'] - real_flow.loc[group, '北/西']) / real_flow.loc[group, '北/西']* 100:.2f}%
    """)

——————————————————————校估104年昏峰指派流量——————————————————————
【SL1】 基隆河
    UE flow 往南/東 = 42069， model diff = 12.74%， CUBE diff = -4.41%
    UE flow 往北/西 = 40036， model diff = 23.41%， CUBE diff = -2.28%
    
【SL2】 台鐵(市民大道)
    UE flow 往南/東 = 37553， model diff = 12.81%， CUBE diff = 7.74%
    UE flow 往北/西 = 30339， model diff = -4.56%， CUBE diff = 4.02%
    
【SL3】 國1-市中心
    UE flow 往南/東 = 4948， model diff = -6.59%， CUBE diff = 0.06%
    UE flow 往北/西 = 5470， model diff = -0.34%， CUBE diff = -19.63%
    
【SL5】 臺北縣市東側運輸走廊
    UE flow 往南/東 = 14910， model diff = -2.55%， CUBE diff = -18.15%
    UE flow 往北/西 = 12244， model diff = 8.95%， CUBE diff = -18.37%
    
【SL6】 北市北側
    UE flow 往南/東 = 2878， model diff = -30.26%， CUBE diff = -5.93%
    UE flow 往北/西 = 5982， model diff = 8.33%， CUBE diff = 9.05%
    
【SL7】 臺北縣市西側運輸走廊(淡水河流域)
    UE flow 往南/東 = 28153， model diff = 11.53%， CUBE diff = 10.24%
    UE flow 往北/西 = 36612， model diff = 20.96%， CUBE diff = 9.83%
    
【SL8】 臺北縣市南側運輸走廊(新店溪&景美溪流域)
    UE flo

In [6]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table

,group,link,UE_flow_1,UE_flow_2
4,SL1,中山橋,1250.561547,421.158369
16,SL1,南湖大橋,1989.508952,2614.019324
5,SL1,大直橋,2307.260075,2292.408717
15,SL1,成功橋,2097.830846,1769.002610
14,SL1,成美橋,1218.237494,1444.325224
2,SL1,承德橋,1651.555299,947.782311
3,SL1,承德橋(機車道),716.666901,982.265139
8,SL1,民權大橋,1792.477582,1684.685320
9,SL1,民權大橋(機車道),1059.038174,1031.641511
0,SL1,洲美快速道路,2089.145828,4429.350428
